In [2]:
# %%bash
import os
os.environ['http_proxy'] = 'http://localhost:8081'
os.environ['https_proxy'] = 'http://localhost:8081'
os.environ['NCCL_P2P_DISABLE'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [3]:
%cd '/mnt/nfs-data/users/anhvth5/projects/guidance/'
%load_ext autoreload
%autoreload 2

/mnt/nfs-data/users/anhvth5/projects/guidance


In [3]:
from speedy import *
from guidance import *

In [4]:
data = load_by_ext('../tvpl_crawler/data/tvpl_qa_short_answer_giaothong/qa_short_answer/*')
questions = []
for qa in data:
    questions.extend(qa['questions'])

2024-01-01 05:05:56.983 | INFO     | speedy.utils:multi_process:262 - Multi processing load_by_ext | Num samples: 6788


  0%|          | 0/6788 [00:00<?, ?it/s]

In [8]:
test_data = load_by_ext('../../data/chat-formated-dataset/tvpl_giaothong_rag_nli_vietnamese_val.json')
gemini_data = load_by_ext('../../data/chat-formated-dataset/giaothong_ask_gemini_suports_or_neutral.json')
qac_data = load_by_ext('../../data/chat-formated-dataset/qac_to_chat_augmentation.json')

In [6]:
from llm_lora.kilm_app import KilmForQAC, KilmForNLI
kilm_qac = KilmForQAC('../kilm-finetuning/output/kilm/01_qac_nli/')

kilm_nli = KilmForNLI('../kilm-finetuning/output/kilm/01_qac_nli/')

2024-01-01 05:06:02.689 | INFO     | llm_lora.load_transformer:<module>:56 - Using interactive mode for `load_transformer_llm`


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
2024-01-01 05:06:21.877 | INFO     | llm_lora.kilm_app:__init__:75 - True id: 14415, False id: 26239
2024-01-01 05:06:21.880 | INFO     | llm_lora.kilm_app:__init__:75 - True id: 14415, False id: 26239


In [9]:
qac_test = load_by_ext('../../data/qac-data/za_qac/qac_test.jsonl')
preds = []
labels = []
for item in tqdm(qac_test):
    pred = kilm_qac(item['question'],'{} | {}'.format(item['title'], item['text']))
    labels.append(item['label'])
    preds.append(pred)
scores = np.array([pred['score'] for pred in preds])

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print('acc', f1_score(labels, scores > 0.35))

  0%|          | 0/3512 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


acc 0.8943965517241379


In [15]:
print('acc', f1_score(labels, scores > 0.5))

acc 0.8947951273532669


In [23]:
from llm_lora.transformer_generator import KilmChatInterface
system = '''Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:

Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.'''

kilm_chat = KilmChatInterface(kilm_nli.model, kilm_nli.tokenizer, system)

In [99]:
print(test_data[3]['conversations'][3]['value'])

Mức độ liên quan (1-5): 5
Tài liệu đã truy xuất hoàn toàn liên quan đến câu hỏi và câu trả lời mong đợi, vì nó cung cấp thông tin chi tiết và cụ thể về chức năng của Tổng giám đốc Tổng công ty Hàng không Việt Nam, phù hợp với Điều lệ tổ chức và hoạt động của Tổng công ty Hàng không Việt Nam.


In [31]:
user = test_data[3]['conversations'][0]['value']
out = kilm_chat.generate('''<|im_start|>system  
Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:

Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.<|im_end|>  
<|im_start|>user  

Câu hỏi (question): ```Tổng giám đốc Tổng công ty Hàng không Việt Nam có chức năng là gì?```

Câu trả lời mong đợi (expected answer): ```Theo Điều 38 Điều lệ tổ chức và hoạt động của Tổng công ty Hàng không Việt Nam, Tổng giám đốc là người đại diện theo pháp luật, điều hành hoạt động hằng ngày của VIETNAM AIRLINES theo mục tiêu, kế hoạch và các nghị quyết, quyết định của Hội đồng thành viên, phù hợp với Điều lệ của VIETNAM AIRLINES và quy định của pháp luật; chịu trách nhiệm trước pháp luật, chủ sở hữu và Hội đồng thành viên về việc thực hiện quyền hạn và nhiệm vụ được giao.
```

Tài liệu đã truy xuất (retrived document): ```Nghị định 183/2013/NĐ-CP Điều lệ tổ chức và hoạt động của Tổng công ty Hàng không Việt Nam mới nhất
Chương 4.
MỤC 2. TỔNG GIÁM ĐỐC |  Điều 38. Chức năng của T ổ ng giám đốc
T ổ ng giám đốc là người đại diện theo pháp luật, điều hành hoạt động hằng ngày của VIETNAM AIRLINES theo mục tiêu, kế hoạch và các nghị quyết, quyết định của Hội đồng thành viên, phù hợp với Điều lệ của VIETNAM AIRLINES và quy định của pháp luật; chịu trách nhiệm trước pháp luật, chủ sở hữu và Hội đồng thành viên về việc thực hiện quyền hạn và nhiệm vụ được giao.```

Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.

<|im_end|>  
<|im_start|>assistant  
Để xác định liệu tài liệu đã truy xuất có hỗ trợ cho câu hỏi và câu trả lời hay không, chúng ta sẽ:

1. Phân Tích Bối Cảnh Tài Liệu: Đánh giá nội dung tổng quan và ngữ cảnh của tài liệu.
2. So Sánh Câu Hỏi và Tài Liệu: Kiểm tra sự tham chiếu trực tiếp hoặc các khái niệm liên quan trong tài liệu phù hợp với câu hỏi.
3. Liên Hệ Câu Trả Lời và Tài Liệu: Xác định xem các yếu tố chính hoặc chi tiết cụ thể của câu trả lời có được hỗ trợ bởi tài liệu không.
4. Đánh Giá Mâu Thuẫn: Xác định bất  kỳ mâu thuẫn nào giữa tài liệu và câu trả lời mong đợi.<|im_end|>
<|im_start|>user
Hãy đánh giá mức độ liên quan trên thang điểm 1-5 cho tài liệu đã truy xuất với câu hỏi và câu trả lời mong đợi.<|im_end|>
<|im_start|>assistant
''', max_new_tokens=1000)

<|im_start|>system   
Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:

Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.<|im_end|>   
<|im_start|>user   

Câu hỏi (question): ```Tổng giám đốc Tổng công ty Hàng không Việt Nam có chức năng là gì?```

Câu trả lời mong đợi (expected answer): ```Theo Điều 38 Điều lệ tổ chức và hoạt động của Tổng công ty Hàng không Việt Nam, Tổng giám đốc là người đại diện theo pháp luật, điều hành hoạt động hằng ngày của VIETNAM AIRLINES theo mục tiêu, kế hoạch và các nghị quyết, quyết định của Hội đồng thành viên, phù hợp với Điều lệ của VIETNAM AIRLINES và quy định của pháp luật; chịu trách nhi

In [25]:
# # item = test_data[6]

# inp = f'''Question: ```{q}```

# Expected answer: ```{a}```

# Retrived document: ``{d}```

# The objective is to determine whether the document supports the question and answer.'''

# assistant_prefix = '''
# To determine if the retrieved document supports the question and answer, we will:

# 1. Analyze Document Context: Evaluate the overall content and context of the retrieved document.
# 2. Match Question and Document: Check for direct references or related concepts in the document that align with the question.
# 3. Correlate Answer and Document: Determine if the key elements or specifics of the expected answer are supported by the document.
# 4. Assess for Contradictions: Identify any contradictions between the document and the expected answer.
# 5. Draw Conclusion: Based on the analysis, conclude if the document supports or contradicts the expected answer.'''

# kilm_chat.chat(inp, system_msg=system, max_new_tokens=1000, assistant_prefix=assistant_prefix);

In [43]:
item = qac_test[13]
q = item['question']
print(q)
a = 'Nhà lê sơ trải qua 3 triều đại: Lê Sơ, Lê Trung Hưng, Lê Hậu Lê với 4 vị vua'
d = '{} | {}'.format(item['title'], item['text'])


nhà lê sơ có mấy vị vua


In [98]:
# kilm_chat.generate(
#     f'''<|im_start|>system   
# Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:

# Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
# Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
# Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
# Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.<|im_end|>   
# <|im_start|>user   

# Câu hỏi (question): ```{q}```

# Expected answer: ```{a}```

# Retrived document: ``{d}```
# Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.

# <|im_start|>assistant 

# Để xác định liệu tài liệu đã truy xuất có hỗ trợ cho câu hỏi và câu trả lời hay không, chúng ta sẽ:

# 1. Phân Tích Bối Cảnh Tài Liệu: Đánh giá nội dung tổng quan và ngữ cảnh của tài liệu.
# 2. So Sánh Câu Hỏi và Tài Liệu: Kiểm tra sự tham chiếu trực tiếp hoặc các khái niệm liên quan trong tài liệu phù hợp với câu hỏi.
# 3. Liên Hệ Câu Trả Lời và Tài Liệu: Xác định xem các yếu tố chính hoặc chi tiết cụ thể của câu trả lời có được hỗ trợ bởi tài liệu không.
# 4. Đánh Giá Mâu Thuẫn: Xác định bất  kỳ mâu thuẫn nào giữa tài liệu và '''
# , max_new_tokens=1000);

In [48]:
llm = models.Kilm(kilm_chat.model, kilm_chat.tokenizer)

2024-01-01 05:23:53.096 | INFO     | guidance.models.transformers._kilm:__init__:48 - Set tokenizer.eos_token_id = tokenizer.eod_id
2024-01-01 05:23:53.097 | INFO     | guidance.models.transformers._qwen:__init__:47 - Set tokenizer.eos_token_id = tokenizer.eod_id


In [101]:
import requests

def search(query, k=3, port=19003):
    url = f"http://localhost:{port}/api/search"
    params = {
        "query": query,
        "return_dict": "true",
        'k': k
    }
    response = requests.get(url, params=params, proxies={'http':'', 'https':''})
    topk = response.json()['topk']
    # print(topk[0].keys())
    # docid_to_text = {t['pid']:f'{t["title"]}| {t["text"]}' for t in topk}
    docid_to_text = [{'doc_id': t['pid'], 'text': f'{t["title"]}| {t["text"]}'} for t in topk]
    return docid_to_text
    
query = 'Cho tôi hỏi: Hồ sơ đề nghị thu hồi, xóa Giấy chứng nhận đăng ký phương tiện giao thông đường sắt được quy định gồm những gì? Văn bản pháp luật nào quy định về vấn đề này?'

search_results = search(query, k=3)

print(json.dumps(search_results, ensure_ascii=False, indent=2))

[
  {
    "doc_id": "620b1ed67b96b50c_1",
    "text": "Hồ sơ đề nghị thu hồi Giấy chứng nhận đăng ký phương tiện giao thông đường sắt gồm những giấy tờ gì? |  Hồ sơ đề nghị thu hồi Giấy chứng nhận đăng ký phương tiện giao thông đường sắt gồm những giấy tờ được căn cứ theo Điều 8 Thông tư 14/2023/TT-BGTVT (Có hiệu lực từ 01/09/2023) sau đây:\nHồ sơ đề nghị thu hồi, xóa Giấy chứng nhận đăng ký phương tiện\nHồ sơ bao gồm bản chính văn bản giấy hoặc bản điện tử đơn đề nghị thu hồi, xóa Giấy chứng nhận đăng ký phương tiện theo mẫu quy định tại Phụ lục VI của Thông tư này; bản gốc hoặc bản điện tử Giấy chứng nhận đăng ký phương tiện đã được cấp.\nNhư vậy, hồ sơ đề nghị thu hồi Giấy chứng nhận đăng ký phương tiện giao thông đường sắt gồm những giấy tờ sau:\n- Mẫu đơn đề nghị thu hồi Giấy chứng nhận đăng ký phương tiện giao thông đường sắt hiện nay đang sử dụng theo mẫu quy định tại Phụ lục VI ban hành kèm theo Thông tư 14/2023/TT-BGTVT (Có hiệu lực từ 01/09/2023).\n- Bản gốc hoặc bản điện tử 

In [105]:
qa = questions[2]
q = qa['question']
# q = 'Cho tôi hỏi: Hồ sơ đề nghị thu hồi, xóa Giấy chứng nhận đăng ký phương tiện giao thông đường sắt được quy định gồm những gì? Văn bản pháp luật nào quy định về vấn đề này?'
a = qa['answer']
search_results = search(q, k=10, )
d = search_results[0]['text']
d

'Chưa thi bằng lái xe mà điều khiển xe gây tai nạn thì có được bồi thường hay không? |  Căn cứ, Điều 584 và Điều 585 Bộ luật dân sự 2015 quy định về căn cứ phát sinh trách nhiệm bồi thường thiệt hại và nguyên tắc bồi thường thiệt hại như sau:\n"Căn cứ phát sinh trách nhiệm bồi thường thiệt hại\n1. Người nào có hành vi xâm phạm tính mạng, sức khỏe, danh dự, nhân phẩm, uy tín, tài sản, quyền, lợi ích hợp pháp khác của người khác mà gây thiệt hại thì phải bồi thường, trừ trường hợp Bộ luật này, luật khác có liên quan quy định khác.\n2. Người gây thiệt hại không phải chịu trách nhiệm bồi thường thiệt hại trong trường hợp thiệt hại phát sinh là do sự kiện bất khả kháng hoặc hoàn toàn do lỗi của bên bị thiệt hại, trừ trường hợp có thỏa thuận khác hoặc luật có quy định khác.\n3. Trường hợp tài sản gây thiệt hại thì chủ sở hữu, người chiếm hữu tài sản phải chịu trách nhiệm bồi thường thiệt hại, trừ trường hợp thiệt hại phát sinh theo quy định tại khoản 2 Điều này.’\nNguyên tắc bồi thường thiệt

In [106]:
inp = f'''<|im_start|><|im_start|>system    
Bạn đã được giao nhiệm vụ Suy Luận Ngôn Ngữ Tự Nhiên (NLI), bao gồm việc phân tích một câu hỏi, câu trả lời đã cho, và tài liệu đã truy xuất. Hãy làm theo các bước sau:

Hiểu Rõ Câu Hỏi: Nắm bắt toàn bộ ngữ cảnh và chi tiết cụ thể.
Phân Tích Câu Trả Lời: Xem xét các điểm chính, lập luận, và chi tiết của nó.
Xem Xét Tài Liệu: Tìm kiếm thông tin liên quan đến câu hỏi và câu trả lời.
Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.<|im_end|>    
<|im_start|>user    

Câu hỏi (question): ``{q}```

Expected answer: ```{a}```

Retrived document: ``{d}```
Mục tiêu là xác định liệu tài liệu có hỗ trợ cho câu hỏi và câu trả lời hay không.

<|im_start|>assistant  

Để xác định liệu tài liệu đã truy xuất có hỗ trợ cho câu hỏi và câu trả lời hay không, chúng ta sẽ:

1. Phân Tích Bối Cảnh Tài Liệu: Đánh giá nội dung tổng quan và ngữ cảnh của tài liệu.
2. So Sánh Câu Hỏi và Tài Liệu: Kiểm tra sự tham chiếu trực tiếp hoặc các khái niệm liên quan trong tài liệu phù hợp với câu hỏi.
3. Liên Hệ Câu Trả Lời và Tài Liệu: Xác định xem các yếu tố chính hoặc chi tiết cụ thể của câu trả lời có được hỗ trợ bởi tài liệu không.
4. Đánh Giá Mâu Thuẫn: Xác định bất  kỳ mâu thuẫn nào giữa tài liệu và  câu trả lời mong đợi.

Bây giờ tôi sẽ thực hiện các bước trên để phân tích tài liệu đã truy xuất.

**Phân Tích Bối Cảnh Tài Liệu**'''

In [107]:
llm+inp+gen(stop='<|im_end|>')+\
    f'<|im_end>\n<|im_start|>user\nHãy đánh giá mức độ liên quan trên thang điểm 1-5, với 5 là hoàn toàn liên quan, 1 là hoàn toàn không liên quan<|im_end|>\n<|im_start|>assistant\nMức độ liên quan (1-5):' + gen(stop='<|im_end|>')

Exception: Attempted to run a transformers model past its maximum context window size of 2048!